# Getting Started with `pyhelios`
***

<img align="right" src="https://github.com/3dgeo-heidelberg/helios/blob/dev/h++.png?raw=true" width="300">

This page will give an introduction on using HELIOS++ python bindings with `pyhelios`. 

`pyhelios` allows you to:

- Access and modify simulation configurations
- Launch one or multiple simulations from your python script
- Read point measurements and process them further in combination with other python modules

## Importing `pyhelios`
The package `pyhelios` is contained in the distribution root folder. It consists of different python scripts and the python-helios bindings. These bindings are actually present in the `run`-folder as `_pyhelios.pyd/[.so]`, but are imported when you import `pyhelios`. The python scripts contain functions to create and work with simulations (e.g., `SimulationBuilder`) and a `util` subpackage, where tools for scene generation and flight planning are found.

Before importing, make sure that the HELIOS++ root directory is either the current working folder or added to the PATH environment variable.
As this notebook is not located in the HELIOS++ root directory, we add the path to the HELIOS++ root directory to the PATH environment variable with `sys.path.append`.

In [1]:
import pyhelios

print(pyhelios.__version__)

0.1.dev1003+g06ef94c.d20240213


In [4]:
import os
os.chdir("..")

## Logging Level and Random Seed

In [5]:
# pyhelios.loggingQuiet()
# pyhelios.loggingSilent()
pyhelios.loggingDefault()
# pyhelios.loggingVerbose()
# pyhelios.loggingVerbose2()

# Set seed for default random number generator.
pyhelios.setDefaultRandomnessGeneratorSeed("123")

## Building a Simulation

In [6]:
simBuilder = pyhelios.SimulationBuilder(
    "data/surveys/toyblocks/als_toyblocks.xml", "assets/", "output/"
)
# simBuilder.setNumThreads(1)  # use only one thread (to ensure reproducibility)
simBuilder.setLasOutput(True)
simBuilder.setZipOutput(True)
simBuilder.setCallbackFrequency(10)  # Run with callback
simBuilder.setFinalOutput(True)  # Return output at join
# simBuilder.setExportToFile(False)  # Disable export point cloud to file
simBuilder.setRebuildScene(True)

sim = simBuilder.build()

SimulationBuilder is building simulation ...
SimulationBuilder built simulation in 0.03192284699980519 seconds
xmlDocFilename: als_toyblocks.xml
xmlDocFilePath: data/surveys/toyblocks
xmlDocFilename: scanners_als.xml
xmlDocFilePath: data
Using default value for attribute 'averagePower_w' : 4
Using default value for attribute 'beamQualityFactor' : 1
Using default value for attribute 'opticalEfficiency' : 0.99
Using default value for attribute 'receiverDiameter_m' : 0.15
Using default value for attribute 'atmosphericVisibility_km' : 23
Scanner: riegl_vq-880g
Device[0]: riegl_vq-880g
	Average Power: 4 W
	Beam Divergence: 0.3 mrad
	Wavelength: 1064 nm
	Visibility: 23 km

Using default value for attribute 'maxNOR' : 0
Using default value for attribute 'rangeMax_m' : 1.79769e+308
Using default value for attribute 'headRotatePerSecMax_deg' : 0
Number of subsampling rays (riegl_vq-880g): 19
xmlDocFilename: platforms.xml
xmlDocFilePath: data
Using default value for attribute 'y' : 0
Using defau

## Starting, Pausing and Getting the Simulation Status

Simulations can also be paused, resumed and stopped:
```python
sim.start()
sim.pause()
sim.resume()
sim.stop()
```

With various functions, we can find out the simulation status.
```python
sim.isStarted()
sim.isRunning()
sim.isPaused()
sim.isStopped()
sim.isFinished()
```

In [7]:
import time

sim.start()

if sim.isStarted():
    print("Simulation is started!")

time.sleep(1.0)
sim.pause()

if sim.isPaused():
    print("Simulation is paused!")

if not sim.isRunning():
    print("Simulation is not running.")

time.sleep(5)
start_time = time.time()
sim.resume()

if sim.isRunning():
    print("Simulation is resumed!")

while sim.isRunning():
    duration = time.time() - start_time
    mins = duration // 60
    secs = duration % 60
    print(
        "\r"
        + "Simulation is running since {} min and {} sec. Please wait.".format(
            int(mins), int(secs)
        ),
        end="",
    )
    time.sleep(1)

if sim.isFinished():
    print("\nSimulation has finished.")

Simulation is started!
Output directory: "output//toyblocks_als/2024-02-13_11-05-56/"
Simulation: Scanner changed!
Starting leg 0

Pulse frequency set to 300000
Scan angle set to 20
Iterative mode was used for manual leg initialization because default one failed for MovingPlatform
Leg0 waypoints:
	Origin: (20, 20, 100.234)
	Target: (120, 20, 100.234)
	Next: (120, 70, 100.234)

Clearing point cloud: "output//toyblocks_als/2024-02-13_11-05-56/leg000_points.laz"
Survey 0.25%	Elapsed 00 00:00:00 Remaining 00 00:00:03
Leg1/5 1.00%	Elapsed 00 00:00:00 Remaining 00 00:00:00
Survey 0.50%	Elapsed 00 00:00:00 Remaining 00 00:00:03
Leg1/5 2.00%	Elapsed 00 00:00:00 Remaining 00 00:00:00
Survey 0.75%	Elapsed 00 00:00:00 Remaining 00 00:00:03
Leg1/5 3.00%	Elapsed 00 00:00:00 Remaining 00 00:00:00
Survey 1.00%	Elapsed 00 00:00:00 Remaining 00 00:00:03
Leg1/5 4.00%	Elapsed 00 00:00:00 Remaining 00 00:00:00
Survey 1.25%	Elapsed 00 00:00:00 Remaining 00 00:00:03
Leg1/5 5.00%	Elapsed 00 00:00:00 Remainin

## Output Handling

If final output was enabled (`simBuilder.setFinalOutput(True)`)., the simulation output, i.e. measurement and trajectory points, can be accessed using `sim.join()`.

In [8]:
# Create instance of PyHeliosOutputWrapper class using sim.join().
# Contains attributes 'measurements' and 'trajectories' which are Python wrappers
# of classes that contain the output vectors.
output = sim.join()

# Create instances of vector classes by accessing 'measurements' and 'trajectories' attributes of output wrapper.
measurements = output.measurements
trajectories = output.trajectories

# Each element of vectors contains a measurement point or point in trajectory respectively.
# Access through getPosition().
starting_point = trajectories[0].getPosition()
end_point = trajectories[len(trajectories) - 1].getPosition()

# Access individual x, y and z vals.
print(
    f"Trajectory starting point : ({starting_point.x}, {starting_point.y}, {starting_point.z})"
)
print(
    f"Trajectory end point : ({end_point.x:.1f}, {end_point.y:.1f}, {end_point.z:.1f})"
)

Trajectory starting point : (-30.0, -50.0, 100.0)
Trajectory end point : (69.9, 50.0, 100.0)


`pyhelios` contains additional tools for output handling (pyhelios/output_handling.py). These allow to convert the trajectory and point outputs to lists or numpy arrays.

In [9]:
meas_array, traj_array = pyhelios.outputToNumpy(output)

In [10]:
import numpy as np

np.set_printoptions(formatter={"float": "{0:0.3f}".format})
print(
    f"""
First three rows of measurement array:

{meas_array[:3, :]}

First three rows of trajectory array:

{traj_array[:3, :]}
"""
)


First three rows of measurement array:

[[-24.944 -13.595 0.012 20.003 20.085 100.994 0.047 0.339 -0.940
  79595.041 0.000 1.000 1.000 33.000 0.000 0.000 209147000106667.000]
 [-29.846 -13.246 0.012 20.000 20.085 100.994 0.001 0.342 -0.940
  76492.360 0.000 1.000 1.000 1.000 0.000 0.000 209147000000000.000]
 [-20.132 -14.596 0.012 20.006 20.085 100.994 0.092 0.329 -0.940
  76281.575 0.000 1.000 1.000 65.000 0.000 0.000 209147000213334.000]]

First three rows of trajectory array:

[[-30.000 -50.000 100.000 209147000000000.000 0.000 0.000 4.712]
 [-29.700 -50.000 100.000 209147010000031.000 0.000 0.000 4.712]
 [-29.400 -50.000 100.000 209147020000062.000 0.000 0.000 4.712]]



Columns of the measurements array:

|    0   |    1   |    2   |    3   |    4   |    5   |    6   |    7   |    8   |    9   |   10   |   11   |   12   |   13   |   14   |   15   |
|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
| pos.x | pos.y | pos.z | ori.x | ori.y | ori.z | dir.x | dir.y | dir.z | intensity | echoWidth | NumberOfReturns | ReturnNumber | FullwaveIndex | classification | gpsTime |


Columns of the trajectories array:


|    0   |    1   |    2   |    3   |    4   |    5   |    6   |
|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
| pos.x | pos.y | pos.z | gpsTime | roll | pitch | yaw |